In [1]:
import torch
# assert '.'.join(torch.__version__.split('.')[:2]) == '1.4'
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F  # useful stateless functions

import numpy as np

In [2]:
from utils import PCamDataset
from three_layers_convnet import ThreeLayerConvNet

In [3]:
USE_GPU = True
dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


In [4]:
def check_accuracy_part34(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=dtype)
            scores = model(x)
            preds = (scores > 0).type("torch.FloatTensor")

            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [5]:
def train_part34(model, optimizer, loader_accuracy, epochs=1, print_every=100):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            scores = model(x)
            
            criterion = torch.nn.BCEWithLogitsLoss()
            loss = criterion(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_accuracy, model)
                print()

In [6]:
size = 100

In [7]:
train_dataset = PCamDataset(csv_file="train_labels.csv", size=size)
val_dataset = PCamDataset(csv_file="dev_labels.csv", size=None)

In [8]:
batch_size = 2

loader_train = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              shuffle=True)
loader_val = DataLoader(val_dataset, 
                              batch_size=batch_size, 
                              shuffle=True)

In [9]:
epochs = 10
learning_rate = 5e-2  # 5e-2 exploding loss
print_every = 10

in_channel = 3
channel_1 = 96
channel_2 = 48

model = ThreeLayerConvNet(in_channel=in_channel, 
                          channel_1=channel_1,
                          channel_2=channel_2)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer, loader_accuracy=loader_train, epochs=epochs, print_every=print_every)

Iteration 0, loss = 211.7878
Got 62 / 100 correct (62.00)

Iteration 10, loss = 0.8800
Got 42 / 100 correct (42.00)

Iteration 20, loss = 0.6932
Got 39 / 100 correct (39.00)

Iteration 30, loss = 0.6935
Got 38 / 100 correct (38.00)

Iteration 40, loss = 0.6944
Got 63 / 100 correct (63.00)

Iteration 0, loss = 0.6238
Got 62 / 100 correct (62.00)

Iteration 10, loss = 0.7004
Got 62 / 100 correct (62.00)

Iteration 20, loss = 0.7003
Got 62 / 100 correct (62.00)

Iteration 30, loss = 0.5824
Got 62 / 100 correct (62.00)

Iteration 40, loss = 0.8004
Got 62 / 100 correct (62.00)

Iteration 0, loss = 0.8543
Got 62 / 100 correct (62.00)

Iteration 10, loss = 0.5487
Got 62 / 100 correct (62.00)

Iteration 20, loss = 0.5250
Got 62 / 100 correct (62.00)

Iteration 30, loss = 0.7151
Got 62 / 100 correct (62.00)

Iteration 40, loss = 0.9395
Got 62 / 100 correct (62.00)

Iteration 0, loss = 0.7104
Got 62 / 100 correct (62.00)

Iteration 10, loss = 0.5125
Got 62 / 100 correct (62.00)

Iteration 20, lo

KeyboardInterrupt: 